In [278]:
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pandas as pd

Выгрузка таблицы *beer_comments* из базы данных *pivo_commets*

In [279]:
import psycopg2
import pandas as pd
from sqlalchemy import create_engine
from psycopg2 import OperationalError

def create_connection(db_name, db_user, db_password, db_host, db_port):
    connection = None
    try:
        connection = psycopg2.connect(
            database=db_name,
            user=db_user,
            password=db_password,
            host=db_host,
            port=db_port
        )
        print("Соединение с базой данных успешно установлено")
    except OperationalError as e:
        print(f"Произошла ошибка '{e}'")
    return connection

def fetch_data_and_save_to_file(engine, file_path, table_name, schema="public"):

    with engine.connect() as connection:
        sql_query = f"SELECT * FROM {schema}.{table_name};"
        df = pd.read_sql(sql_query, connection)

        df.to_csv(file_path, index=False)
        print(f"Данные сохранены в файл {file_path}")

db_params = {
    "db_name": "pivo_commets",
    "db_user": "postgres",
    "db_password": "1111",
    "db_host": "127.0.0.1",
    "db_port": "5432"
}
file_path = "comments_train.csv"
table_name = "beer_comments"

conn = create_connection(**db_params)
cursor = conn.cursor()


db_uri = f"postgresql://{db_params['db_user']}:{db_params['db_password']}@{db_params['db_host']}:{db_params['db_port']}/{db_params['db_name']}"
engine = create_engine(db_uri)

try:
    fetch_data_and_save_to_file(engine, file_path, table_name)

except KeyboardInterrupt:
    print("Скрипт завершен")
finally:
    cursor.close()
    conn.close()


Соединение с базой данных успешно установлено
Данные сохранены в файл comments_train.csv


In [280]:
comments_train = pd.read_csv(r"F:\beeeer\gan_comment\comments_train.csv")
comments_train

,id,text,beer_rating
0,1,"После просмотра этой дорамы я впервые подумал,...",9.0
1,2,"Алкоголь в этой дораме – не просто напиток, а ...",8.0
2,3,Главные герои этой дорамы – настоящие ценители...,9.5
3,4,Пивная тема в этой дораме раскрыта на высшем у...,9.0
4,5,Эта дорама – настоящая ржачка! Персонажи знают...,8.5
...,...,...,...
97,137,"Легкий сериал, интересно посмотреть, чтобы отд...",4.0
98,138,Реалистично. Жёстко. Актёры - супер. Очень жаль.,5.0
99,139,Фильм сильный!!! Очень тяжёлая и несправедлива...,5.0
100,140,"Реалистичный, без прикрас. Как есть, все показ...",4.0


In [281]:
comments_train_copy = comments_train.copy()

Удаляем эмоджи

In [282]:
import emoji
import re
import pandas as pd

def remove_emoji(input_text):
    cleaned_text = re.sub(r'(:[^:\s]+:|[\U00010000-\U0010ffff])', '', input_text)
    return cleaned_text

comments_train_copy = comments_train_copy.applymap(lambda x: remove_emoji(str(x)))

print(comments_train_copy.head())


  id                                               text beer_rating
0  1  После просмотра этой дорамы я впервые подумал,...         9.0
1  2  Алкоголь в этой дораме – не просто напиток, а ...         8.0
2  3  Главные герои этой дорамы – настоящие ценители...         9.5
3  4  Пивная тема в этой дораме раскрыта на высшем у...         9.0
4  5  Эта дорама – настоящая ржачка! Персонажи знают...         8.5


In [283]:
text = ''.join(comments_train_copy['text'].astype(str).tolist())

vocab = sorted(set(text))
char2idx = {u: i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)
text_as_int = np.array([char2idx[c] for c in text])

In [284]:
vocab_size = len(vocab)
print("Vocabulary size:", vocab_size)
print("Encoded text size:", len(text_as_int))

Vocabulary size: 90
Encoded text size: 17801


In [285]:
seq_length = 100
examples_per_epoch = len(text) // (seq_length + 1)

char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

sequences = char_dataset.batch(seq_length + 1, drop_remainder=True)

for item in sequences.take(5):
    print(repr(''.join(idx2char[item.numpy()])))

'После просмотра этой дорамы я впервые подумал, что пиво может стать главным персонажем. Алкогольная и'
'нтрига, угарные сцены – просто ржака! Под пиво идеально!Алкоголь в этой дораме – не просто напиток, а'
' настоящий проводник к угару и радостному безумию. Под пиво просматривается легко, рекомендую!Главные'
' герои этой дорамы – настоящие ценители пива. Их приключения просто угарные, и каждая сцена – как под'
' пиво, так и под сильный алкоголь!Пивная тема в этой дораме раскрыта на высшем уровне. Алкогольные пр'


In [286]:
BATCH_SIZE = 1
BUFFER_SIZE = 10000
seq_length = 100
examples_per_epoch = len(text_as_int) // (seq_length + 1)

char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)
sequences = char_dataset.batch(seq_length + 1, drop_remainder=True)

vocab_size = len(vocab)
embedding_dim = 256
rnn_units = 1024
rnn_units_2 = 512

In [287]:
dataset = sequences.map(split_input_target)
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

In [288]:
import tensorflow as tf
import os

checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, batch_input_shape=[BATCH_SIZE, None]),
    tf.keras.layers.GRU(rnn_units, return_sequences=True, stateful=True, recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dropout(0.2),  # Добавлен слой Dropout с коэффициентом отсева 0.2
    tf.keras.layers.Dense(vocab_size)
])

def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

model.compile(optimizer='adam', loss=loss)

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True
)

latest_checkpoint = tf.train.latest_checkpoint(checkpoint_dir)
if latest_checkpoint is not None:
    model.load_weights(latest_checkpoint)

epochs = 10

history = model.fit(dataset, epochs=epochs, callbacks=[checkpoint_callback])



Epoch 1/10


176/176 [==============================] - 125s 692ms/step - loss: 0.2403
Epoch 2/10
176/176 [==============================] - 119s 675ms/step - loss: 0.3315
Epoch 3/10
176/176 [==============================] - 118s 672ms/step - loss: 0.3292
Epoch 4/10
176/176 [==============================] - 119s 675ms/step - loss: 0.2754
Epoch 5/10
176/176 [==============================] - 119s 674ms/step - loss: 0.2316
Epoch 6/10
176/176 [==============================] - 120s 681ms/step - loss: 0.2037
Epoch 7/10
176/176 [==============================] - 119s 674ms/step - loss: 0.1767
Epoch 8/10
176/176 [==============================] - 123s 699ms/step - loss: 0.1720
Epoch 9/10
176/176 [==============================] - 126s 714ms/step - loss: 0.1726
Epoch 10/10
176/176 [==============================] - 119s 678ms/step - loss: 0.1683


In [289]:
if model.weights:
    print("Модель содержит веса.")
else:
    print("Модель не содержит весов.")

Модель содержит веса.


In [290]:
model.summary()

Model: "sequential_20"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_17 (Embedding)    (1, None, 256)            23040     
                                                                 
 gru_28 (GRU)                (1, None, 1024)           3938304   
                                                                 
 dropout_1 (Dropout)         (1, None, 1024)           0         
                                                                 
 dense_26 (Dense)            (1, None, 90)             92250     
                                                                 
Total params: 4053594 (15.46 MB)
Trainable params: 4053594 (15.46 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [291]:
model.save('model_after_generation_0.h5')
model.save_weights('model_after_weights.h5')

C:\Users\Ульяна\AppData\Roaming\Python\Python311\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [292]:
start_string = "эта дорама "
for _ in range(10):
    generated_text = generate_text(model, start_string=start_string, temp=0.5, gen_chars=200)
    print(generated_text)
    print("\n" + "="*50 + "\n")  # Добавить разделитель между комментариями


эта дорама с пуренаселением была бы ещё более серьёзной, как бы нам не хотелось, что бы таких людеко было интересно чем же всё закончится. Смотря триллер представляла что-то поэпичнее, сразу после просмотреть. О


эта дорама скучна. Это повод для праздника. Алкоголь не спасает от неинтересных персонажей и нудно и ужасно скучно...жаль потраченного временифильм тронул до костяшек. очень душевно и искренне, не раскаят просмо


эта дорама скучная, как стакан воды. Под пиво не стоит тратить время.Эта дорама - настоящий пивной рай! Алкогольные забавы персонажей наполняют каждую серию смехом и угаром. Под пиво смотрится легко и весело!Дор


эта дорама с пивом. Под пиво смотрится легко и весело!Дорама, где пиво – самое то!Герои этой дорамы просто мостегом, что делает просмотр под пиво смотрится легко и весело!Дорама, где пиво – самое то!Герои этой д


эта дорама с его участием и уже восхищаюсь актёрским мастерством. в наше время не редко бывают угарами смотрут смотреть что-то про психопатов с 